In [45]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import os
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine, exc
from dotenv import load_dotenv

load_dotenv()

True

In [48]:
start_date = '2023-01-01'
end_date = '2024-06-30'
etf = 'QQQ'

### Gather Stock data through Polygon.ai and clean it using Pandas df

* Convert date from unix timestamp to datetime object
* Rename Columns     

In [49]:

def polygon(stock_ticker, date_start, date_end):

    # Set up the GET request
    polygon_url = 'https://api.polygon.io/v2/aggs/ticker'
    stocksTicker = f'/{stock_ticker}'
    timespan = '/range/1/month'
    date_from = f'/{date_start}'
    date_to = f'/{date_end}'
    params = '?adjusted=true&sort=asc&'
    polygon_key = os.environ['polygon_api_key']

    params = {'adjusted':'true', 
            'sort': 'asc',
            'apiKey': polygon_key
            }

    full_url = polygon_url + stocksTicker + timespan + date_from + date_to

    # Request the data
    response2 = requests.get(full_url, params)
    polygon_data = response2.json()

    # Access the "results" part of the response
    results = polygon_data["results"]
    
    df_polygon = pd.DataFrame(results)
    df_polygon.head(5)

    # Convert the Unix timestamp to datetime
    df_polygon['date'] = pd.to_datetime(df_polygon['t'], unit='ms')

    # Format the datetime to (YYYY-MM-DD)
    df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m-%d')

    # Drop the unix time column
    df_polygon = df_polygon.drop(columns=['t'])

    # Rename cols
    df_polygon.rename(columns={'v': 'volume', 'o':'open', 'c':'close', 'h':'high', 'l':'low'}, inplace=True)

    #drop unused cols
    df_polygon = df_polygon[['date', 'open', 'high', 'low', 'close']]
    
    # Convert date from object to datetime
    df_polygon[['date']] = df_polygon[['date']].apply(pd.to_datetime)
    #df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m')
    #df_polygon['date'] = df_polygon['date'] + '-01'
    
    
    return df_polygon


### Gather Stock data through AlphaVantage (second source to validate the data) and clean it using Pandas df

* Filter the entries to only include dates from 01/01/2023 to 07/03/2024
* sort by date in asc order
* Convert values from object to numerics

In [50]:
def alpha_vantage(stock_ticker, date_start, date_end):

    # Set up GET request
    base_url = 'https://www.alphavantage.co'
    stocks_endpoint = '/query'

    stocks_url = base_url+stocks_endpoint

    api_key = os.environ['alphavantage_api_key']

    params = {'function':'TIME_SERIES_MONTHLY', 
            'symbol': stock_ticker,
            'outputsize': 'full',
            'apikey' : api_key}

    # Request the data        
    alpha_response = requests.get(stocks_url, params)
    alpha_data = alpha_response.json()
    time_series = alpha_data["Monthly Time Series"] #was Time Series (Daily)

    # Filter the entries to only include dates from 01/01/2023 to 07/03/2024
    start_date = datetime.strptime(date_start, "%Y-%m-%d")
    end_date = datetime.strptime(date_end, "%Y-%m-%d")
    filtered_data = {date: values for date, values in time_series.items() if (datetime.strptime(date, "%Y-%m-%d") >= start_date and datetime.strptime(date, "%Y-%m-%d") <= end_date)}
    
    # Switch cols and rows
    df_alpha = pd.DataFrame.from_dict(filtered_data, orient="index")
    df_alpha.index = pd.to_datetime(df_alpha.index)
    # Rename columns to remove the numeric prefix
    df_alpha = df_alpha.rename(columns=lambda x: x.split(". ")[1])  

    # Reset the index to create a new numerical index
    df_alpha = df_alpha.reset_index()
    df_alpha.rename(columns={'index': 'date'}, inplace=True)

    # Sort DataFrame by 'date' column in ascending order
    df_alpha = df_alpha.sort_values(by='date', ascending=True)
    df_alpha = df_alpha.reset_index()

    df_alpha = df_alpha.drop(columns=['index'])
    
    # Convert values from object to numerics
    df_alpha[['open', 'high', 'low', 'close']] = df_alpha[['open', 'high', 'low', 'close']].apply(pd.to_numeric, errors='coerce')
    
    # Remove 'volume'
    df_alpha = df_alpha[['date', 'open', 'high', 'low', 'close']]
    
    df_alpha['date'] = df_alpha['date'].dt.strftime('%Y-%m')
    df_alpha['date'] = df_alpha['date'] + '-01'
    

    return df_alpha
 

## Data Validation
- Summary statistics
- Comparing to other source (Alpha Vantage)

### Check for Null and NaN Values through summary statistics

In [51]:
# Check for nulls
def null_check(df):
    null_counts = df.isnull().sum()
    return(null_counts)

In [52]:
df_1 = polygon(etf, start_date, end_date)
df_2 = alpha_vantage(etf, start_date, end_date)

print(null_check(df_1))
print('________________________')
print(null_check(df_2))


date     0
open     0
high     0
low      0
close    0
dtype: int64
________________________
date     0
open     0
high     0
low      0
close    0
dtype: int64



### Compare the dataframes obtained
* If the data matches, load it to our SQL database
* If some of the data doesn't match, replace it with the average of both dfs, then store in SQL database

In [53]:
# Check if DataFrames from polygon and alphavantage are equal
if df_1.equals(df_2):
    print("DataFrames are identical.")
else:
    print("DataFrames are different.")
    # Compare DataFrames and filter differences
    mask = df_1 != df_2
    diff_df = df_1[mask.any(axis=1)]  # Select rows where any difference exists
    diff_df2 = df_2[mask.any(axis=1)]  # Select rows where any difference exists

    print("Differences:")
    print(diff_df)
    print('_________________________________________')
    print(diff_df2)

    print('')
    # Calculate the average where differences are found
    avg_values = (df_1[mask] + df_2[mask]) / 2

    # Replace the differing values in both DataFrames with the average values
    df_1[mask] = avg_values
    df_2[mask] = avg_values
    
    # Print new values 
    modified_rows_1 = df_1[mask.any(axis=1)]

    # Print the modified rows
    print("Modified rows:")
    print(modified_rows_1)


DataFrames are different.
Differences:
         date    open    high     low  close
9  2023-10-01  426.62  438.14  408.91  418.2
_________________________________________
         date    open    high     low  close
9  2023-10-01  426.62  438.14  409.21  418.2

Modified rows:
         date    open    high     low  close
9  2023-10-01  426.62  438.14  409.06  418.2


# Load the data to the db

In [54]:
df_1['symbol'] = etf
df_2['symbol'] = etf

df_2.head(5)

,date,open,high,low,close,symbol
0,2023-01-01,384.370,408.16,377.831,406.48,SPY
1,2023-02-01,405.211,418.31,393.640,396.26,SPY
2,2023-03-01,395.410,409.70,380.650,409.39,SPY
3,2023-04-01,408.850,415.94,403.780,415.93,SPY
4,2023-05-01,415.470,422.58,403.740,417.85,SPY


In [55]:
# Define the connection string for Windows Authentication
server = 'MoemenLaptop'
database = 'InvestmentPortfolio'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establish the connection
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")
    cursor = conn.cursor()

    # Iterate over rows of the DataFrame
    for index, row in df_2.iterrows():
        # Example SQL insert command
        sql_command = "INSERT INTO Monthly_Data (date, [open], high, low, [close], symbol) VALUES (?, ?, ?, ?, ?, ?)"
        
        # Execute the command
        cursor.execute(sql_command, tuple(row))

    # Commit the transaction
    conn.commit()
    
    print("Records Inserted!")

except Exception as e:
    print(f"Error: {e}")
    
    
conn.close()

Connection successful!
Records Inserted!
